## 金融风控项目
在此项目中，你需要完成金融风控模型的搭建。 基于一个用户的基本信息、历史信息来预测逾期与否。采样的具体数据是拍拍贷数据。https://www.kesci.com/home/competition/56cd5f02b89b5bd026cb39c9/content/1
在此数据中提供了三种不同类型的数据:
1. Master: 用户的主要信息
2. Loginfo: 登录信息
3. Userupdateinfo: 修改信息

在本次项目中，我们只使用```Master```的信息来预测一个用户是否会逾期。 数据里有一个字段叫作```Target```是样本的标签（label)。 在```Master```表格里，包含200多个特征，但不少特征具有缺失值。 做项目的时候需要仔细处理一下。 

对于特征处理方面的技术，请参考本章视频课程里的内容。

本项目区别于之前的项目，希望大家能够开放性地思考，不要太局限于给定的条条框框，把目前为止学到的知识都用起来。所以在项目的设计上区别于之前的，没有设置太多的框架性的，大家可以按照自己的思路灵活做项目。 由于项目本身来自于数据竞赛，所以可以试着跟竞赛里的TOP选手的结果做一下对比，看看跟他们的差距或者优势在哪儿。

```数据```
- ```Training/PPD_Training_Master_GBK_3_1_Training_Set.csv```:  训练数据
- ```Test/PPD_Master_GBK_2_Test_Set.csv```: 测试数据


强调：一定要把注释写清楚。 每个函数，每一个模块具体做什么写清楚。

> 注意： 除了下面导入的库，还有sklearn、XGBoost等经典的库之外，建议不要使用其他的函数库。 如果你不得不使用一些其他特殊的库，请把环境注明在requirements.txt里面，不然我们判作业的时候就没有办法去评判了。 

In [414]:
import numpy as np 
import math 
import pandas as pd 
pd.set_option('display.float_format',lambda x:'%.3f' % x)
import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline
import seaborn as sns 
sns.set_palette('muted')
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')
import os 

pd.options.display.max_columns = None
from scipy import stats

In [415]:
! ls ./data/Training/

PPD_LogInfo_3_1_Training_Set.csv
PPD_Training_Master_GBK_3_1_Training_Set.csv
PPD_Userupdate_Info_3_1_Training_Set.csv


In [416]:
# 读取Master数据
data = pd.read_csv('data/Training/PPD_Training_Master_GBK_3_1_Training_Set.csv',encoding='gb18030')
print (data.shape)

(30000, 228)


In [417]:
# 展示记录
data.head()

,Idx,UserInfo_1,UserInfo_2,UserInfo_3,UserInfo_4,WeblogInfo_1,WeblogInfo_2,WeblogInfo_3,WeblogInfo_4,WeblogInfo_5,WeblogInfo_6,WeblogInfo_7,WeblogInfo_8,WeblogInfo_9,WeblogInfo_10,WeblogInfo_11,WeblogInfo_12,WeblogInfo_13,WeblogInfo_14,WeblogInfo_15,WeblogInfo_16,WeblogInfo_17,WeblogInfo_18,UserInfo_5,UserInfo_6,UserInfo_7,UserInfo_8,UserInfo_9,UserInfo_10,UserInfo_11,UserInfo_12,UserInfo_13,UserInfo_14,UserInfo_15,UserInfo_16,UserInfo_17,UserInfo_18,UserInfo_19,UserInfo_20,UserInfo_21,UserInfo_22,UserInfo_23,UserInfo_24,Education_Info1,Education_Info2,Education_Info3,Education_Info4,Education_Info5,Education_Info6,Education_Info7,Education_Info8,WeblogInfo_19,WeblogInfo_20,WeblogInfo_21,WeblogInfo_23,WeblogInfo_24,WeblogInfo_25,WeblogInfo_26,WeblogInfo_27,WeblogInfo_28,WeblogInfo_29,WeblogInfo_30,WeblogInfo_31,WeblogInfo_32,WeblogInfo_33,WeblogInfo_34,WeblogInfo_35,WeblogInfo_36,WeblogInfo_37,WeblogInfo_38,WeblogInfo_39,WeblogInfo_40,WeblogInfo_41,WeblogInfo_42,WeblogInfo_43,WeblogInfo_44,WeblogInfo_45,WeblogInfo_46,WeblogInfo_47,WeblogInfo_48,WeblogInfo_49,WeblogInfo_50,WeblogInfo_51,WeblogInfo_52,WeblogInfo_53,WeblogInfo_54,WeblogInfo_55,WeblogInfo_56,WeblogInfo_57,WeblogInfo_58,ThirdParty_Info_Period1_1,ThirdParty_Info_Period1_2,ThirdParty_Info_Period1_3,ThirdParty_Info_Period1_4,ThirdParty_Info_Period1_5,ThirdParty_Info_Period1_6,ThirdParty_Info_Period1_7,ThirdParty_Info_Period1_8,ThirdParty_Info_Period1_9,ThirdParty_Info_Period1_10,ThirdParty_Info_Period1_11,ThirdParty_Info_Period1_12,ThirdParty_Info_Period1_13,ThirdParty_Info_Period1_14,ThirdParty_Info_Period1_15,ThirdParty_Info_Period1_16,ThirdParty_Info_Period1_17,ThirdParty_Info_Period2_1,ThirdParty_Info_Period2_2,ThirdParty_Info_Period2_3,ThirdParty_Info_Period2_4,ThirdParty_Info_Period2_5,ThirdParty_Info_Period2_6,ThirdParty_Info_Period2_7,ThirdParty_Info_Period2_8,ThirdParty_Info_Period2_9,ThirdParty_Info_Period2_10,ThirdParty_Info_Period2_11,ThirdParty_Info_Period2_12,ThirdParty_Info_Period2_13,ThirdParty_Info_Period2_14,ThirdParty_Info_Period2_15,ThirdParty_Info_Period2_16,ThirdParty_Info_Period2_17,ThirdParty_Info_Period3_1,ThirdParty_Info_Period3_2,ThirdParty_Info_Period3_3,ThirdParty_Info_Period3_4,ThirdParty_Info_Period3_5,ThirdParty_Info_Period3_6,ThirdParty_Info_Period3_7,ThirdParty_Info_Period3_8,ThirdParty_Info_Period3_9,ThirdParty_Info_Period3_10,ThirdParty_Info_Period3_11,ThirdParty_Info_Period3_12,ThirdParty_Info_Period3_13,ThirdParty_Info_Period3_14,ThirdParty_Info_Period3_15,ThirdParty_Info_Period3_16,ThirdParty_Info_Period3_17,ThirdParty_Info_Period4_1,ThirdParty_Info_Period4_2,ThirdParty_Info_Period4_3,ThirdParty_Info_Period4_4,ThirdParty_Info_Period4_5,ThirdParty_Info_Period4_6,ThirdParty_Info_Period4_7,ThirdParty_Info_Period4_8,ThirdParty_Info_Period4_9,ThirdParty_Info_Period4_10,ThirdParty_Info_Period4_11,ThirdParty_Info_Period4_12,ThirdParty_Info_Period4_13,ThirdParty_Info_Period4_14,ThirdParty_Info_Period4_15,ThirdParty_Info_Period4_16,ThirdParty_Info_Period4_17,ThirdParty_Info_Period5_1,ThirdParty_Info_Period5_2,ThirdParty_Info_Period5_3,ThirdParty_Info_Period5_4,ThirdParty_Info_Period5_5,ThirdParty_Info_Period5_6,ThirdParty_Info_Period5_7,ThirdParty_Info_Period5_8,ThirdParty_Info_Period5_9,ThirdParty_Info_Period5_10,ThirdParty_Info_Period5_11,ThirdParty_Info_Period5_12,ThirdParty_Info_Period5_13,ThirdParty_Info_Period5_14,ThirdParty_Info_Period5_15,ThirdParty_Info_Period5_16,ThirdParty_Info_Period5_17,ThirdParty_Info_Period6_1,ThirdParty_Info_Period6_2,ThirdParty_Info_Period6_3,ThirdParty_Info_Period6_4,ThirdParty_Info_Period6_5,ThirdParty_Info_Period6_6,ThirdParty_Info_Period6_7,ThirdParty_Info_Period6_8,ThirdParty_Info_Period6_9,ThirdParty_Info_Period6_10,ThirdParty_Info_Period6_11,ThirdParty_Info_Period6_12,ThirdParty_Info_Period6_13,ThirdParty_Info_Period6_14,ThirdParty_Info_Period6_15,ThirdParty_Info_Period6_16,ThirdParty_Info_Period6_17,ThirdParty_Info_Period7_1,ThirdParty_Info_Period7_2,ThirdParty_Info_Period7_3,ThirdParty_Info_Perio

In [418]:
# 正负样本的比例， 可以看出样本比例不平衡的
data.target.value_counts(0)

0    27802
1     2198
Name: target, dtype: int64

### <font color=blue> Step 1： 因为target =1 太少，先oversample  10倍 target =1 的数据加入到原来的30000行数据里进行training

In [419]:
data_y1=pd.concat([data[data.target==1]]*10,ignore_index=True)
print (len(data_y1))

21980


In [420]:
data=data.append(data_y1)
print (data.shape)

(51980, 228)


In [421]:
data.target.value_counts()

0    27802
1    24178
Name: target, dtype: int64

好了，剩下的部分需要由大家完成。 我大致给一下思路，然后大家可以按照这个思路去一步步完成。 

> #### 1. 数据的预处。 需要考虑以下几个方面：
- ```缺失值```。数据里有大量的缺失值，需要做一些处理。 
- ```字符串的清洗```。比如“北京市”和“北京”合并成“北京”， 统一转换成小写等。 在数据中有一些关于城市的数，而且城市本身对于逾期率还是有着很强的相关性。
- ```二值化```。具体方法请参考课程里的介绍：对城市二值化的处理。
- ```衍生特征```：比如户籍地和当前城市是否是同一个？ 
- ```特征的独热编码```：对于类别型特征使用独热编码形式
- ```连续性特征的处理```：根据情况来处理
- ```其他```: 根据情况，自行决定要不要做
- data dictionary https://www.kesci.com/home/competition/56cd5f02b89b5bd026cb39c9/content/1

In [422]:
describe=data.describe()

In [423]:
describe

,Idx,UserInfo_1,UserInfo_3,WeblogInfo_1,WeblogInfo_2,WeblogInfo_3,WeblogInfo_4,WeblogInfo_5,WeblogInfo_6,WeblogInfo_7,WeblogInfo_8,WeblogInfo_9,WeblogInfo_10,WeblogInfo_11,WeblogInfo_12,WeblogInfo_13,WeblogInfo_14,WeblogInfo_15,WeblogInfo_16,WeblogInfo_17,WeblogInfo_18,UserInfo_5,UserInfo_6,UserInfo_10,UserInfo_11,UserInfo_12,UserInfo_13,UserInfo_14,UserInfo_15,UserInfo_16,UserInfo_17,UserInfo_18,UserInfo_21,Education_Info1,Education_Info5,WeblogInfo_23,WeblogInfo_24,WeblogInfo_25,WeblogInfo_26,WeblogInfo_27,WeblogInfo_28,WeblogInfo_29,WeblogInfo_30,WeblogInfo_31,WeblogInfo_32,WeblogInfo_33,WeblogInfo_34,WeblogInfo_35,WeblogInfo_36,WeblogInfo_37,WeblogInfo_38,WeblogInfo_39,WeblogInfo_40,WeblogInfo_41,WeblogInfo_42,WeblogInfo_43,WeblogInfo_44,WeblogInfo_45,WeblogInfo_46,WeblogInfo_47,WeblogInfo_48,WeblogInfo_49,WeblogInfo_50,WeblogInfo_51,WeblogInfo_52,WeblogInfo_53,WeblogInfo_54,WeblogInfo_55,WeblogInfo_56,WeblogInfo_57,WeblogInfo_58,ThirdParty_Info_Period1_1,ThirdParty_Info_Period1_2,ThirdParty_Info_Period1_3,ThirdParty_Info_Period1_4,ThirdParty_Info_Period1_5,ThirdParty_Info_Period1_6,ThirdParty_Info_Period1_7,ThirdParty_Info_Period1_8,ThirdParty_Info_Period1_9,ThirdParty_Info_Period1_10,ThirdParty_Info_Period1_11,ThirdParty_Info_Period1_12,ThirdParty_Info_Period1_13,ThirdParty_Info_Period1_14,ThirdParty_Info_Period1_15,ThirdParty_Info_Period1_16,ThirdParty_Info_Period1_17,ThirdParty_Info_Period2_1,ThirdParty_Info_Period2_2,ThirdParty_Info_Period2_3,ThirdParty_Info_Period2_4,ThirdParty_Info_Period2_5,ThirdParty_Info_Period2_6,ThirdParty_Info_Period2_7,ThirdParty_Info_Period2_8,ThirdParty_Info_Period2_9,ThirdParty_Info_Period2_10,ThirdParty_Info_Period2_11,ThirdParty_Info_Period2_12,ThirdParty_Info_Period2_13,ThirdParty_Info_Period2_14,ThirdParty_Info_Period2_15,ThirdParty_Info_Period2_16,ThirdParty_Info_Period2_17,ThirdParty_Info_Period3_1,ThirdParty_Info_Period3_2,ThirdParty_Info_Period3_3,ThirdParty_Info_Period3_4,ThirdParty_Info_Period3_5,ThirdParty_Info_Period3_6,ThirdParty_Info_Period3_7,ThirdParty_Info_Period3_8,ThirdParty_Info_Period3_9,ThirdParty_Info_Period3_10,ThirdParty_Info_Period3_11,ThirdParty_Info_Period3_12,ThirdParty_Info_Period3_13,ThirdParty_Info_Period3_14,ThirdParty_Info_Period3_15,ThirdParty_Info_Period3_16,ThirdParty_Info_Period3_17,ThirdParty_Info_Period4_1,ThirdParty_Info_Period4_2,ThirdParty_Info_Period4_3,ThirdParty_Info_Period4_4,ThirdParty_Info_Period4_5,ThirdParty_Info_Period4_6,ThirdParty_Info_Period4_7,ThirdParty_Info_Period4_8,ThirdParty_Info_Period4_9,ThirdParty_Info_Period4_10,ThirdParty_Info_Period4_11,ThirdParty_Info_Period4_12,ThirdParty_Info_Period4_13,ThirdParty_Info_Period4_14,ThirdParty_Info_Period4_15,ThirdParty_Info_Period4_16,ThirdParty_Info_Period4_17,ThirdParty_Info_Period5_1,ThirdParty_Info_Period5_2,ThirdParty_Info_Period5_3,ThirdParty_Info_Period5_4,ThirdParty_Info_Period5_5,ThirdParty_Info_Period5_6,ThirdParty_Info_Period5_7,ThirdParty_Info_Period5_8,ThirdParty_Info_Period5_9,ThirdParty_Info_Period5_10,ThirdParty_Info_Period5_11,ThirdParty_Info_Period5_12,ThirdParty_Info_Period5_13,ThirdParty_Info_Period5_14,ThirdParty_Info_Period5_15,ThirdParty_Info_Period5_16,ThirdParty_Info_Period5_17,ThirdParty_Info_Period6_1,ThirdParty_Info_Period6_2,ThirdParty_Info_Period6_3,ThirdParty_Info_Period6_4,ThirdParty_Info_Period6_5,ThirdParty_Info_Period6_6,ThirdParty_Info_Period6_7,ThirdParty_Info_Period6_8,ThirdParty_Info_Period6_9,ThirdParty_Info_Period6_10,ThirdParty_Info_Period6_11,ThirdParty_Info_Period6_12,ThirdParty_Info_Period6_13,ThirdParty_Info_Period6_14,ThirdParty_Info_Period6_15,ThirdParty_Info_Period6_16,ThirdParty_Info_Period6_17,ThirdParty_Info_Period7_1,ThirdParty_Info_Period7_2,ThirdParty_Info_Period7_3,ThirdParty_Info_Period7_4,ThirdParty_Info_Period7_5,ThirdParty_Info_Period7_6,ThirdParty_Info_Period7_7,ThirdParty_Info_Period7_8,ThirdParty_Info_Period7_9,ThirdParty_Info_Period7_10,ThirdParty_Info_Period7_11,ThirdParty_Info_Period7_12,ThirdParty_Info_Period7_13,ThirdParty

In [424]:
na_count=describe[describe.index=='count'].T.reset_index()
na_count.head(10)

,index,count
0,Idx,51980.000
1,UserInfo_1,51974.000
2,UserInfo_3,51973.000
3,WeblogInfo_1,1790.000
4,WeblogInfo_2,49022.000
5,WeblogInfo_3,1790.000
6,WeblogInfo_4,49029.000
7,WeblogInfo_5,49029.000
8,WeblogInfo_6,49029.000
9,WeblogInfo_7,51980.000


### <font color=blue>  先看一下有多少variable 有很多missing value

In [425]:
na_count['count'].describe(percentiles=[0.01,0.02,0.025,0.03,0.04,0.05,0.95,0.99])

count     208.000
mean    50899.750
std      6343.773
min      1790.000
1%      17841.000
2%      22206.340
2.5%    49023.225
3%      49029.000
4%      49750.840
5%      51607.000
50%     51980.000
95%     51980.000
99%     51980.000
max     51980.000
Name: count, dtype: float64

In [ ]:
#median=describe[describe.index=='50%'].T.reset_index()
#median[median['50%']==-1]
#for i in median[median['50%']==-1]['index']:
    #del data[i]
#print (data.shape)

### <font color=blue> Step 2： 因为注意到很多ThirdParty Info 的continuous variable 都是 -1 这个值，我的估计是 这个-1 应该是第三方机构用来标注 missing value 的代码， 所以我决定把这些-1 全部用 median 进行取代，并create 一个新的 dummy variabe 叫做_imputed 去记录这一行的这个value 到底是不是impute 得来的

In [426]:
describe=data.describe()
min_is_negative1=describe[describe.index=='min'].T.reset_index()
print (min_is_negative1[min_is_negative1['min']==-1]['index'])
continus_replace_list=min_is_negative1[min_is_negative1['min']==-1]['index']
print (len(continus_replace_list))

71      ThirdParty_Info_Period1_1
72      ThirdParty_Info_Period1_2
73      ThirdParty_Info_Period1_3
74      ThirdParty_Info_Period1_4
75      ThirdParty_Info_Period1_5
76      ThirdParty_Info_Period1_6
77      ThirdParty_Info_Period1_7
78      ThirdParty_Info_Period1_8
79      ThirdParty_Info_Period1_9
80     ThirdParty_Info_Period1_10
81     ThirdParty_Info_Period1_11
82     ThirdParty_Info_Period1_12
83     ThirdParty_Info_Period1_13
84     ThirdParty_Info_Period1_14
85     ThirdParty_Info_Period1_15
86     ThirdParty_Info_Period1_16
87     ThirdParty_Info_Period1_17
88      ThirdParty_Info_Period2_1
89      ThirdParty_Info_Period2_2
90      ThirdParty_Info_Period2_3
91      ThirdParty_Info_Period2_4
92      ThirdParty_Info_Period2_5
93      ThirdParty_Info_Period2_6
94      ThirdParty_Info_Period2_7
95      ThirdParty_Info_Period2_8
96      ThirdParty_Info_Period2_9
97     ThirdParty_Info_Period2_10
98     ThirdParty_Info_Period2_11
99     ThirdParty_Info_Period2_12
100    ThirdPa

In [427]:
def replace_negative1(df, col):
    imputed=str(col+'_imputed')
    df[imputed]=0
    median = df.loc[df[col]!=-1.0, col].median()
    df[imputed]=np.where(df[col]==-1.0, 1, 0)
    df[col] = np.where(df[col]==-1.0, median,df[col])
    return df 
#print (data['ThirdParty_Info_Period1_1'].describe())
#data=replace_negative1(data, 'ThirdParty_Info_Period1_1')
#print (data['ThirdParty_Info_Period1_1'].describe())

In [428]:
for i in continus_replace_list:
    print (i)
    #print (data[i].describe())
    data=replace_negative1(data, i)
    #print (data[i].describe())

ThirdParty_Info_Period1_1
ThirdParty_Info_Period1_2
ThirdParty_Info_Period1_3
ThirdParty_Info_Period1_4
ThirdParty_Info_Period1_5
ThirdParty_Info_Period1_6
ThirdParty_Info_Period1_7
ThirdParty_Info_Period1_8
ThirdParty_Info_Period1_9
ThirdParty_Info_Period1_10
ThirdParty_Info_Period1_11
ThirdParty_Info_Period1_12
ThirdParty_Info_Period1_13
ThirdParty_Info_Period1_14
ThirdParty_Info_Period1_15
ThirdParty_Info_Period1_16
ThirdParty_Info_Period1_17
ThirdParty_Info_Period2_1
ThirdParty_Info_Period2_2
ThirdParty_Info_Period2_3
ThirdParty_Info_Period2_4
ThirdParty_Info_Period2_5
ThirdParty_Info_Period2_6
ThirdParty_Info_Period2_7
ThirdParty_Info_Period2_8
ThirdParty_Info_Period2_9
ThirdParty_Info_Period2_10
ThirdParty_Info_Period2_11
ThirdParty_Info_Period2_12
ThirdParty_Info_Period2_13
ThirdParty_Info_Period2_14
ThirdParty_Info_Period2_15
ThirdParty_Info_Period2_16
ThirdParty_Info_Period2_17
ThirdParty_Info_Period3_1
ThirdParty_Info_Period3_2
ThirdParty_Info_Period3_3
ThirdParty_Info_Period

In [429]:
data[continus_replace_list].describe()

,ThirdParty_Info_Period1_1,ThirdParty_Info_Period1_2,ThirdParty_Info_Period1_3,ThirdParty_Info_Period1_4,ThirdParty_Info_Period1_5,ThirdParty_Info_Period1_6,ThirdParty_Info_Period1_7,ThirdParty_Info_Period1_8,ThirdParty_Info_Period1_9,ThirdParty_Info_Period1_10,ThirdParty_Info_Period1_11,ThirdParty_Info_Period1_12,ThirdParty_Info_Period1_13,ThirdParty_Info_Period1_14,ThirdParty_Info_Period1_15,ThirdParty_Info_Period1_16,ThirdParty_Info_Period1_17,ThirdParty_Info_Period2_1,ThirdParty_Info_Period2_2,ThirdParty_Info_Period2_3,ThirdParty_Info_Period2_4,ThirdParty_Info_Period2_5,ThirdParty_Info_Period2_6,ThirdParty_Info_Period2_7,ThirdParty_Info_Period2_8,ThirdParty_Info_Period2_9,ThirdParty_Info_Period2_10,ThirdParty_Info_Period2_11,ThirdParty_Info_Period2_12,ThirdParty_Info_Period2_13,ThirdParty_Info_Period2_14,ThirdParty_Info_Period2_15,ThirdParty_Info_Period2_16,ThirdParty_Info_Period2_17,ThirdParty_Info_Period3_1,ThirdParty_Info_Period3_2,ThirdParty_Info_Period3_3,ThirdParty_Info_Period3_4,ThirdParty_Info_Period3_5,ThirdParty_Info_Period3_6,ThirdParty_Info_Period3_7,ThirdParty_Info_Period3_8,ThirdParty_Info_Period3_9,ThirdParty_Info_Period3_10,ThirdParty_Info_Period3_11,ThirdParty_Info_Period3_12,ThirdParty_Info_Period3_13,ThirdParty_Info_Period3_14,ThirdParty_Info_Period3_15,ThirdParty_Info_Period3_16,ThirdParty_Info_Period3_17,ThirdParty_Info_Period4_1,ThirdParty_Info_Period4_2,ThirdParty_Info_Period4_3,ThirdParty_Info_Period4_4,ThirdParty_Info_Period4_5,ThirdParty_Info_Period4_6,ThirdParty_Info_Period4_7,ThirdParty_Info_Period4_8,ThirdParty_Info_Period4_9,ThirdParty_Info_Period4_10,ThirdParty_Info_Period4_11,ThirdParty_Info_Period4_12,ThirdParty_Info_Period4_13,ThirdParty_Info_Period4_14,ThirdParty_Info_Period4_15,ThirdParty_Info_Period4_16,ThirdParty_Info_Period4_17,ThirdParty_Info_Period5_1,ThirdParty_Info_Period5_2,ThirdParty_Info_Period5_3,ThirdParty_Info_Period5_4,ThirdParty_Info_Period5_5,ThirdParty_Info_Period5_6,ThirdParty_Info_Period5_7,ThirdParty_Info_Period5_8,ThirdParty_Info_Period5_9,ThirdParty_Info_Period5_10,ThirdParty_Info_Period5_11,ThirdParty_Info_Period5_12,ThirdParty_Info_Period5_13,ThirdParty_Info_Period5_14,ThirdParty_Info_Period5_15,ThirdParty_Info_Period5_16,ThirdParty_Info_Period5_17,ThirdParty_Info_Period6_1,ThirdParty_Info_Period6_2,ThirdParty_Info_Period6_3,ThirdParty_Info_Period6_4,ThirdParty_Info_Period6_5,ThirdParty_Info_Period6_6,ThirdParty_Info_Period6_7,ThirdParty_Info_Period6_8,ThirdParty_Info_Period6_9,ThirdParty_Info_Period6_10,ThirdParty_Info_Period6_11,ThirdParty_Info_Period6_12,ThirdParty_Info_Period6_13,ThirdParty_Info_Period6_14,ThirdParty_Info_Period6_15,ThirdParty_Info_Period6_16,ThirdParty_Info_Period6_17,ThirdParty_Info_Period7_1,ThirdParty_Info_Period7_2,ThirdParty_Info_Period7_3,ThirdParty_Info_Period7_4,ThirdParty_Info_Period7_5,ThirdParty_Info_Period7_6,ThirdParty_Info_Period7_7,ThirdParty_Info_Period7_8,ThirdParty_Info_Period7_9,ThirdParty_Info_Period7_10,ThirdParty_Info_Period7_11,ThirdParty_Info_Period7_12,ThirdParty_Info_Period7_13,ThirdParty_Info_Period7_14,ThirdParty_Info_Period7_15,ThirdParty_Info_Period7_16,ThirdParty_Info_Period7_17,SocialNetwork_3,SocialNetwork_4,SocialNetwork_5,SocialNetwork_6,SocialNetwork_7,SocialNetwork_8,SocialNetwork_9,SocialNetwork_10,SocialNetwork_11,SocialNetwork_12
count,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,51980.000,

### <font color=blue> Step 3： 如果一个特征有超过50% 以上的样本上都是NAN，就删掉这个特征。一共删掉了5个variables

In [430]:
print ('Data has ', len(data.columns), ' variables.')
for i in na_count[na_count['count']<51980*0.5]['index']:
    print (i, 'been deleted from the dataset.')
    del data[i]
print ('Data has ', len(data.columns), ' variables after drop columns with too many missing values.')

Data has  357  variables.
WeblogInfo_1 been deleted from the dataset.
WeblogInfo_3 been deleted from the dataset.
UserInfo_11 been deleted from the dataset.
UserInfo_12 been deleted from the dataset.
UserInfo_13 been deleted from the dataset.
Data has  352  variables after drop columns with too many missing values.


### <font color=blue> Step 4：假如一个样本包含了超过50%以上的值为NAN，可以删掉此样本。 但是发现并没有那么多样本有大量值都是NAN，原因是大量的variable 的nan其实都用特殊的代码来代替了，比如’不详‘， -1 等，所以暂且保留所有样本， 争取把missing value 编码成特殊的一类，然后在做模型的最后一步统一删除所有有missing value 的 observation

In [431]:
data['row_none_na_count'] = data.apply(lambda x: x.count(), axis=1)

In [432]:
data['row_none_na_count'].describe(percentiles=[0.01,0.02,0.03,0.04,0.05])

count   51980.000
mean      351.123
std         2.679
min       318.000
1%        345.000
2%        345.000
3%        347.000
4%        347.000
5%        348.000
50%       352.000
max       352.000
Name: row_none_na_count, dtype: float64

In [433]:
data.shape

(51980, 353)

### <font color=blue> Step 5：把所有city column 里面的NAN 都换成"不详"

In [434]:
def replace_nan(df, col):
    print (data[col].count())
    data[col]=data[col].apply(lambda x: '不详' if pd.isnull(x)==True else x)
    print (data[col].count())
    return data

In [435]:
data=replace_nan(data, 'UserInfo_2')
data=replace_nan(data, 'UserInfo_4')
data=replace_nan(data, 'UserInfo_7')
data=replace_nan(data, 'UserInfo_8')
data=replace_nan(data, 'UserInfo_20')

51528
51980
51562
51980
51980
51980
51980
51980
51980
51980


### <font color=blue> Step 6： 对所有city column 里面名字都进行同样的清理

In [436]:
def clean_city(df, col):
    print (len(df[col].unique()))
    df[col]=df[col].apply(lambda x: x.replace('市','').replace('地区','').replace('、眉山','')
                                         .replace('彝族自治州','').replace(' 海拉尔','').replace('白族自治州','')
                                        .replace('巴音郭楞蒙古自治州-','').replace('朝鲜族自治州-延吉','').replace('傣族景颇族自治州','')
                                        .replace('土家族苗族自治州','').replace('壮族苗族自治州','').replace('昌吉回族自治州-','')
                                        .replace('哈尼族','').replace('傣族自治州','').replace('藏族自治州','').replace('、嘉峪关','')
                                        .replace('锡林郭勒盟 ','').replace('藏族羌族自治州-马尔康','').replace('苗族侗族自治州','')
                                        .replace('黔南布依族苗族自治州-','').replace('布依族苗族自治州-兴义市',''))
    print (len(df[col].unique()))
    return df

In [437]:
data=clean_city(data, 'UserInfo_2')
data=clean_city(data, 'UserInfo_4')
data=clean_city(data, 'UserInfo_7')
data=clean_city(data, 'UserInfo_8')
data=clean_city(data, 'UserInfo_20')

328
328
331
331
32
32
655
367
297
297


### <font color=blue> Step 7：衍生特征：查看户籍地和当前城市是否是同一个 same city？ 因为data dictionary 非常简略也没有写清楚到底那一列是户籍地，哪一列是当前城市，所以决定把所有和城市相关的的5个columns 做两两编码，然后用chisqure test 保留那些和target 有significant 的衍生特征，一共保留了8个

In [438]:
def same_city(df, col1, col2, output_col):
    df[output_col]=df.apply(lambda x: 1 if x[col1]==x[col2]  else 0, axis=1 )
    p_value=stats.chi2_contingency(df.groupby('target')[output_col].value_counts().unstack().fillna(0))[1]
    if p_value<=0.05:
        print ('p-value of chisquare test for ', output_col, ' is ' , str(p_value))
    else:
        del df[output_col]
    return df

#2, 4, 7, ,8 20
data=same_city(data, 'UserInfo_2', 'UserInfo_4', 'same_city_2_4')
data=same_city(data, 'UserInfo_2', 'UserInfo_7', 'same_city_2_7')
data=same_city(data, 'UserInfo_2', 'UserInfo_8', 'same_city_2_8')
data=same_city(data, 'UserInfo_2', 'UserInfo_20', 'same_city_2_20')
data=same_city(data, 'UserInfo_4', 'UserInfo_7', 'same_city_4_7')
data=same_city(data, 'UserInfo_4', 'UserInfo_8', 'same_city_4_8')
data=same_city(data, 'UserInfo_4', 'UserInfo_20', 'same_city_4_20')
data=same_city(data, 'UserInfo_7', 'UserInfo_8', 'same_city_7_8')
data=same_city(data, 'UserInfo_7', 'UserInfo_20', 'same_city_7_20')
data=same_city(data, 'UserInfo_8', 'UserInfo_20', 'same_city_8_20')

p-value of chisquare test for  same_city_2_4  is  2.113183402083574e-31
p-value of chisquare test for  same_city_2_8  is  0.020939932510807866
p-value of chisquare test for  same_city_2_20  is  3.539619720469828e-25
p-value of chisquare test for  same_city_4_8  is  6.2364883149487736e-06
p-value of chisquare test for  same_city_4_20  is  3.123453914310101e-27
p-value of chisquare test for  same_city_7_8  is  0.008189038696027086
p-value of chisquare test for  same_city_7_20  is  0.03573164807342742
p-value of chisquare test for  same_city_8_20  is  1.0147292328968271e-10


### <font color=blue> Step 8： 对UserInfo_9 进行清理

In [439]:
data['UserInfo_9']=data['UserInfo_9'].apply(lambda x: x.replace(' ',''))
data['UserInfo_9'].value_counts()

中国移动    29572
中国联通     9390
不详       7573
中国电信     5445
Name: UserInfo_9, dtype: int64

> #### ` 二值化`:把和target 没有关系的城市都编码成’其他‘

### <font color=blue> Step 9： 用chisquare test 找出categorical variables 中和 target 有significant (p<=0.05) 关系的类别，其他和target 无关的类别都code 成’其他'

In [440]:
def find_sig_group(df, col):
    sig_list=[]
    for i in df[col].unique():
        try:
            #print (i)
            a=df[(df[col]==i) & (df['target']==0)].target.count()
            b=df[(df[col]==i) & (df['target']==1)].target.count()
            c=df[(df[col]!=i) & (df['target']==0)].target.count()
            d=df[(df[col]!=i) & (df['target']==1)].target.count()
            #print (a, b, c, d)
            obs = np.array( [[a, b],    [c, d]])
            p= stats.chi2_contingency(obs)[1]
            if p<=0.05:
                #print (i)
                sig_list.append(i)
        except Exception:
            pass
    #print (sig_list)
    df[col]=data[col].apply(lambda x: x if x in sig_list else '其他')
    return df 

#data['x']=data['UserInfo_2'].apply(lambda x: x if x in sig_city_list2 else '其他')

In [441]:
cat_columns = ['UserInfo_2','UserInfo_4', 'UserInfo_7','UserInfo_8','UserInfo_9','UserInfo_19','UserInfo_20','UserInfo_22','UserInfo_23',
       'Education_Info2','Education_Info3','Education_Info4','Education_Info6','Education_Info7','Education_Info8',
              'WeblogInfo_19','WeblogInfo_20','WeblogInfo_21']
for x in cat_columns:
    data=find_sig_group(data, x)

### <font color=blue> Step 10： 将categorical variable 的不同value group 之后，删除这些只有unique value的columns， 删除了一个

In [442]:
print (data.shape)
for i in data.columns:
    unique_cnt=len(data[i].unique())
    if unique_cnt==1:
        print (i)
        del data[i]
print (data.shape)

(51980, 361)
WeblogInfo_10
(51980, 360)


In [443]:
#这一列和省份一列有重复， 而且大部分为D 于是删除
del data['UserInfo_24']

In [444]:
print (data.columns)

Index(['Idx', 'UserInfo_1', 'UserInfo_2', 'UserInfo_3', 'UserInfo_4',
       'WeblogInfo_2', 'WeblogInfo_4', 'WeblogInfo_5', 'WeblogInfo_6',
       'WeblogInfo_7',
       ...
       'SocialNetwork_12_imputed', 'row_none_na_count', 'same_city_2_4',
       'same_city_2_8', 'same_city_2_20', 'same_city_4_8', 'same_city_4_20',
       'same_city_7_8', 'same_city_7_20', 'same_city_8_20'],
      dtype='object', length=359)


In [445]:
data[['UserInfo_2','UserInfo_4','UserInfo_7', 'UserInfo_8','UserInfo_20']].head(10)

,UserInfo_2,UserInfo_4,UserInfo_7,UserInfo_8,UserInfo_20
0,深圳,深圳,其他,其他,南充
1,温州,温州,浙江,温州,不详
2,宜昌,宜昌,湖北,宜昌,其他
3,南平,南平,福建,南平,不详
4,辽阳,其他,辽宁,其他,其他
5,吴忠,银川,不详,不详,不详
6,绵阳,其他,内蒙古,其他,其他
7,其他,其他,其他,广州,其他
8,赤峰,其他,内蒙古,其他,其他
9,其他,鄂州,湖北,鄂州,鄂州


idx：每一笔贷款的unique key，可以与另外2个文件里的idx相匹配。

UserInfo_*：借款人特征字段

WeblogInfo_*：Info网络行为字段

Education_Info*：学历学籍字段

ThirdParty_Info_PeriodN_*：第三方数据时间段N字段

SocialNetwork_*：社交网络字段

LinstingInfo：借款成交时间

> #### 2. 特征选择
200多个特征里可能有效的特征不会很多。在这里做特征选择相关的工作。 在特征选择这一块请使用```树```模型。 比如sklearn自带的特征选择模块（https://scikit-learn.org/stable/modules/feature_selection.html）， 或者直接使用XGBoost等模型来直接选择。 这些模型训练好之后你可以直接通过```feature_importance_values```属性来获取。

### <font color=blue> Step 11：对每一个categorical特征实现独热编码转换

In [446]:
clean_cat_columns = ['UserInfo_2','UserInfo_4', 'UserInfo_7','UserInfo_8','UserInfo_9','UserInfo_19','UserInfo_20','UserInfo_22','UserInfo_23',
       'Education_Info2','Education_Info3','Education_Info4','Education_Info6','Education_Info7','Education_Info8',
              'WeblogInfo_19','WeblogInfo_20','WeblogInfo_21']
data= pd.get_dummies(data, prefix_sep="__",columns=clean_cat_columns)

In [447]:
del data['Idx']

In [448]:
del data['ListingInfo']
print (len(data))
print (len(data.dropna()))
data=data.dropna()

51980
48636


In [449]:
y=data.target
del data['target']
X=data

### <font color=blue> Step 12：把dataset 分成两部分，归一化，train 和test 防止overfit， 最后给的test data_set 没有target 所以用不了

In [450]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [452]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(36477, 1158) (12159, 1158) (36477,) (12159,)


In [453]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# 利用StandardScaler进行归一化
X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)

### <font color=blue> Step 13：选一下variables 从1158个X 降到238个X

In [462]:
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X_train, y_train)
#clf.feature_importances_  

In [463]:
print (X_train.shape,  X_test.shape) 

(36477, 1158) (12159, 1158)


In [464]:
clf.feature_importances_.shape

(1158,)

In [465]:
model = SelectFromModel(clf, prefit=True)

In [466]:
X_train_new = model.transform(X_train)
X_test_new = model.transform(X_test)
print (X_train_new.shape,  X_test_new.shape)             

(36477, 238) (12159, 238)


> #### 3. XGBoost来训练风控模型，结果以AUC为准
https://github.com/dmlc/xgboost   这是XGBoost library具体的地址, 具有详细的文档。 https://pypi.org/project/xgboost/ 里有安装的步骤。 试着去调一下它的超参数，使得得到最好的效果。 一定要注意不需要使用测试数据来训练。 最终的结果以测试数据上的AUC为标准。 

### <font color=blue> Step 14：先做一个XGBoost的 baseline model：

In [480]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
model=XGBClassifier().fit(X_train_new, y_train)
import sklearn.metrics as metrics
y_train_predict=model.predict(X_train_new)
print (y_train_predict.sum())
y_test_predict=model.predict(X_test_new)
print (y_test_predict.sum())

16385
5460


#### performance of baseline XGBoost model: 还没有调hyperparamter

In [481]:
import sklearn.metrics as metrics
fpr, tpr, thresholds = metrics.roc_curve(
        y_train, y_train_predict, pos_label=1)
auc = "%.4f" % metrics.auc(fpr, tpr)
print ('AUC based on train dataset is ', auc)

AUC based on train dataset is  0.7539


In [482]:
fpr, tpr, thresholds = metrics.roc_curve(
        y_test, y_test_predict, pos_label=1)
auc = "%.4f" % metrics.auc(fpr, tpr)
print ('AUC based on test dataset is ', auc)

AUC based on test dataset is  0.7365


double check confusion matrix

In [483]:
from sklearn.metrics import confusion_matrix
print (confusion_matrix(y_train_predict,y_train))

[[15391  4701]
 [ 4188 12197]]


In [484]:
from sklearn.metrics import confusion_matrix
print (confusion_matrix(y_test_predict,y_test))

[[4987 1712]
 [1465 3995]]


In [471]:
#import sklearn
#sklearn.metrics.SCORERS.keys()

### <font color=blue> Step 14： gridsearch 找一下更好的参数

In [485]:
parameter_grid= {
        'subsample': [0.1, 0.5,  1.0],
        'colsample_bytree': [0.1, 0.5, 1.0],
        'max_depth': [ 5, 10, 20, 30]
        }
grid_search = GridSearchCV(model,
                           param_grid=parameter_grid,
                           cv=4, n_jobs=5, scoring='roc_auc')
grid_search.fit(X_train_new, y_train)
# 输出最好的参数 
print('Best parameters: {}'.format(grid_search.best_params_))

Best parameters: {'colsample_bytree': 0.1, 'max_depth': 30, 'subsample': 1.0}


In [488]:
y_train_predict=grid_search.predict(X_train_new)
print (y_train_predict.sum())
y_test_predict=grid_search.predict(X_test_new)
print (y_test_predict.sum())

16898
5716


In [489]:
fpr, tpr, thresholds = metrics.roc_curve(
        y_train, y_train_predict, pos_label=1)
auc = "%.4f" % metrics.auc(fpr, tpr)
print ('AUC of train dataset from the gridsearch Xgboost model  is ', auc)

AUC of train dataset from the gridsearch Xgboost model  is  1.0000


In [490]:
fpr, tpr, thresholds = metrics.roc_curve(
        y_test, y_test_predict, pos_label=1)
auc = "%.4f" % metrics.auc(fpr, tpr)
print ('AUC of test dataset from the gridsearch Xgboost model  is', auc)

AUC of test dataset from the gridsearch Xgboost model  is 0.9993


In [491]:
print (confusion_matrix(y_train_predict,y_train))

[[19579     0]
 [    0 16898]]


In [492]:
print (confusion_matrix(y_test_predict,y_test))

[[6443    0]
 [   9 5707]]


### <font color=blue> 结果： 最后我自己split 出来的test dataset 上AUC 是0.9993，只错了9个case？ 感觉有点过高，但是回顾以上步骤，看不出太大的问题

> 
我做这个project 并没有用到下面这个test dataset， 因为下面这个test dataset 中是没有target 这个column的，所以即使我算出predict的值，也没有办法算AUC，accuracy。这个dataset 应该是竞赛的时候最后打分的dataset，主办方比较选手提交的预测结果和实际结果，选出最好的结果作为优胜组。但是我这里并没有正确答案，所以也就无法打分（除非老师这里有这个dataset的target，可以打分，不然处理这个数据也就没什么意义？

所以我目前没有再花时间去处理这个数据， 要把这个数据完全整理成X_train 的样子还是很麻烦的，有些步骤是没办法没办法套用上面已有的的流程，还要重新写code。 

如果老师有这个data_test 的label， 我可以在考虑 测一下这个data上我的model 的AUC 有多少，不然的话我觉得处理这个数据没有意义。

In [360]:
data_test = pd.read_csv('data/Test/PPD_Master_GBK_2_Test_Set.csv',encoding='gb18030')

In [371]:
data_test.shape

(19999, 227)

<font color=red>问题：
1） 看了一下这个比赛的排行榜： https://www.kesci.com/home/competition/56cd5f02b89b5bd026cb39c9/leaderboard <br>    
第一名AUC是：0.79085333， 我目前按照上面步骤在我自己split 出来的25%的test dataset 里面AUC 是 0.9993， 结果貌似好的有点吓人？
我的猜测是因为我的train 和test 都是对target =1 进行oversample的 所以出来的 AUC应该会比较大，而比赛队伍用的上面的这个data_test应该target=1很少，所以估计predict 出来的 AUC就会比较低。  我这样子的理解对不对？<br>
2） 老师还有没有其他的什么建议？<br>
3） 从来没打过比赛，老师对打比赛有什么看法？

0.9993的确很高，可在真正的test效果怎么无人得知，毕竟比赛入口关闭了，建议就是多参考别人的https://github.com/wikke/ppdai_risk_evaluation ，老师没有花时间投入到这个比赛中，肯定不如他们好，比赛是实战，远比做课程里的项目好些，毕竟课程大多不够深入。以后工作面试也多看的是比赛。